<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Data" data-toc-modified-id="Data-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Data</a></span></li><li><span><a href="#Helper-Functions-(Will-be-migrated-to-utils)" data-toc-modified-id="Helper-Functions-(Will-be-migrated-to-utils)-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Helper Functions (Will be migrated to utils)</a></span></li><li><span><a href="#Performance-Comparison" data-toc-modified-id="Performance-Comparison-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Performance Comparison</a></span><ul class="toc-item"><li><span><a href="#Oversampling" data-toc-modified-id="Oversampling-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Oversampling</a></span></li><li><span><a href="#Under-&amp;-Oversampling" data-toc-modified-id="Under-&amp;-Oversampling-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Under &amp; Oversampling</a></span></li></ul></li><li><span><a href="#Analysis" data-toc-modified-id="Analysis-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Analysis</a></span></li></ul></div>

# Oversampling

In [1]:
# from imblearn.combine import SMOTEENN, SMOTETomek
# Environment Setup
from utils import *
from setup import *

Progress: [####################] 100.0%
Added 1048 patients


## Data

In [2]:
train_data

NameError: name 'train_data' is not defined

In [ ]:
train_data = pd.read_csv('train_data.csv')
test_data = pd.read_csv('test_data.csv')

train_pids = train_data['Unnamed: 0'].values
test_pids = test_data['Unnamed: 0'].values

train_y = train_data['BIRADS'].values
train_y_bi = np.where(train_y>3,1,0)
train_data = train_data.drop(columns=['BIRADS', 'Unnamed: 0'])

test_y = test_data['BIRADS'].values
test_y_bi = np.where(test_y>3,1,0)
test_data = test_data.drop(columns=['BIRADS', 'Unnamed: 0'])

In [ ]:
np.sum(test_y_bi)/len(test_y_bi)

## Helper Functions (Will be migrated to utils)

In [ ]:
def plot_tsne(X, y):
    """
    Perform t-sne using the resampled data (can be done
    without augmented data) and blind data.
    """
    x = X.values
    min_max_scaler = preprocessing.MinMaxScaler()
    x_scaled = min_max_scaler.fit_transform(x)
    X_norm = pd.DataFrame(x_scaled)

    tsne = TSNE(n_components=2,
                 init='random',
                 learning_rate=5,
                 random_state=1,
                 perplexity=30,
                 early_exaggeration=20,
                 n_iter=2000,
                 metric='euclidean',
                 angle=0.2)

    y_tsne = tsne.fit_transform(X_norm)


    green = y == 0
    red = y == 1

    plt.scatter(y_tsne[green, 0], y_tsne[green, 1], c="g", alpha = 0.3, label='Healthy')
    plt.scatter(y_tsne[red, 0], y_tsne[red, 1], c="r", alpha = 0.5, label='Cancer')
    plt.axis('off')
    plt.legend()
    plt.show()
    
def plot_tsne_birads(X, y, X_blind, y_blind, train_mean, train_std):
    """
    Perform t-sne using the resampled data (can be done
    without augmented data) and blind data.
    """
    X_norm = (X - train_mean)/train_std
    X_blind_norm = (X_blind - train_mean)/train_std
    
    y_stack = np.append(y, y_blind)
    one = y_stack == 1
    two = y_stack == 2
    three = y_stack == 3
    four = y_stack == 4
    five = y_stack == 5
    six = y_stack == 6
    unlabelled= y_stack == -1

    tsne = TSNE(n_components=2,
                 init='random',
                 learning_rate=5,
                 random_state=1,
                 perplexity=30,
                 early_exaggeration=20,
                 n_iter=2000,
                 metric='euclidean',
                 angle=0.2)

    y_tsne = tsne.fit_transform(np.vstack([X_norm, X_blind_norm]))
    plt.scatter(y_tsne[one, 0], y_tsne[one, 1], c="g", alpha = 0.3, label='1')
    plt.scatter(y_tsne[two, 0], y_tsne[two, 1], c="g", alpha = 0.3, label='2')
    plt.scatter(y_tsne[three, 0], y_tsne[three, 1], c="yellow", alpha = 0.5, label='3')
    plt.scatter(y_tsne[four, 0], y_tsne[four, 1], c="magenta", alpha = 0.5, label='4')
    plt.scatter(y_tsne[five, 0], y_tsne[five, 1], c="blue", alpha = 0.5, label='5')
    plt.scatter(y_tsne[six, 0], y_tsne[six, 1], c="red", alpha = 0.5, label='6')
    plt.scatter(y_tsne[unlabelled, 0], y_tsne[unlabelled, 1], c="g", alpha = 0.1, label='-1')
    plt.legend()
    plt.axis('off')
    plt.show()

In [ ]:
# from sklearn.linear_model import LogisticRegression
def performance(X, y, X_test, y_test, detailed = False, title=''):
    
    """ Obtain tuned LogReg model classification performance.
    

    Inputs
    ----------
    X: dataframe or numpy array
         training set with features
    y: np.array
         training targets    
    X_test: dataframe or numpy array
         test set with features
    y_test: np.array
         test targets    

    detailed: bool
         Indicates whether to print a detailed
         report and ROC curve or not
    title: str
         A title that is given to the ROC
         curve plot and the AUC score
        

    Returns
    -------
    AUC score for a specific technique
    Optional
    """
# Use basic logreg instead of pretrained model
#     exec(compile(open('src/models/classifiers/LogReg.py').read(), 
#                  'src/models/classifiers/LogReg.py', 'exec'), globals())
    clf = LogReg.model
    # Make sure I dont use pre-tuned model for feature evaluation
#     print('Tuned parameters: ', clf.get_params())
    clf.fit(X, y)
    probs = clf.predict_proba(X_test)
    fpr, tpr, thresholds = roc_curve(y_test, probs)
#     print("AUC score: " + title, auc(fpr,tpr))
    if detailed: 
        plt.plot(fpr, tpr)

        plt.xlabel("False Positive Rate")
        plt.ylabel("True Positive Rate (Recall)")
        plt.title(title)
        plt.show()
        report_performance(y_test, probs)
    return(auc(fpr,tpr))


def plot_AUC(aucs, title=''):
    '''Plot a set of AUC scores.

    Inputs
    ----------
    aucs: list
         list of AUC scores
    max_comp: int
         maximum number of components 
    title: str
         A title that is given to the plot
        

    Returns
    -------
    Plot of AUC values
    Maximum AUC score
    '''
    plt.plot(np.arange(0.1, 1.0, 0.1),aucs)
    plt.xlabel('Float Value')
    plt.ylabel('AUC Score')
    plt.title(title)
    plt.show()
    print('Max AUC: ', max(aucs))

    
def plot_AUCs(method, X, X_test, y_bi, y_t, label):
    '''Plot a set of AUC scores for a specific set.

    Inputs
    ----------
    max_comp: int
         maximum number of components 
    X_train: dataframe or numpy array
         training set with reduced features
    X_test: dataframe or numpy array
         test set with reduced features
    y_train: np.array
         training targets    
    y_test: np.array
         test targets  
    label: str
         A title that is given to the plot
    best: bool
        Whether to pick features with best t-values
    stats: dataframe
        Output of the get_stats function
        

    Output
    -------
    Plot of AUC values
    Maximum AUC score
    '''
    avg_base = [performance(X, y_bi, X_test, y_t) for i in range(100)]
    aucs = [np.mean(avg_base)]
    for i in np.arange(0.2, 1.0, 0.1):
        sampler = method(i)
        avg_auc = []
        for i in range(100):
            X_res, y_res = sampler.fit_resample(X, y_bi)
            avg_auc.append(performance(X_res, y_res, X_test, y_t))
        aucs.append(np.mean(avg_auc))
    plot_AUC(aucs, label)

    

## Performance Comparison
### Oversampling

In [ ]:
from imblearn import over_sampling
plot_AUCs(over_sampling.ADASYN, train_data, test_data, train_y_bi, test_y_bi, label='ADASYN')
plot_AUCs(over_sampling.SMOTE, train_data, test_data, train_y_bi, test_y_bi, label='SMOTE')
plot_AUCs(over_sampling.RandomOverSampler, train_data, test_data, train_y_bi, test_y_bi, label='Random')
# These two methods seem to fail
plot_AUCs(over_sampling.SVMSMOTE, train_data, test_data, train_y_bi, test_y_bi, label='SVMSMOTE')
plot_AUCs(over_sampling.BorderlineSMOTE, train_data, test_data, train_y_bi, test_y_bi,label='BorderlineSMOTE')
# No clusters found with sufficient samples of class 1.0 to perform these methods
plot_AUCs(over_sampling.KMeansSMOTE, train_data, test_data, train_y_bi, test_y_bi, label='KMeansSMOTE')
plot_AUCs(over_sampling.SMOTENC, train_data, test_data, train_y_bi, test_y_bi, label='SMOTENC')

### Under & Oversampling

In [ ]:
plot_AUCs(SMOTETomek, train_data, test_data, train_y_bi, test_y_bi,label='SMOTE & Tomek')
# Doesn't work with the data
# plot_AUCs(SMOTEENN, X, X_test, y_bi, y_t, label='SMOTE & ENN')

## Analysis

In [ ]:
# Color by center
plot_tsne(train_data, train_y_bi)

In [ ]:
sampler = over_sampling.ADASYN(0.5)
X_res, y_res = sampler.fit_resample(train_data, train_y_bi)
X_res = pd.DataFrame(X_res)
plot_tsne(X_res, y_res)

In [ ]:
sampler = over_sampling.SMOTE(0.5)
X_res, y_res = sampler.fit_resample(train_data, train_y_bi)
X_res = pd.DataFrame(X_res)
plot_tsne(X_res, y_res)

In [ ]:
sampler = over_sampling.ADASYN(0.5)
X_res, y_res = sampler.fit_resample(X, y_bi)
plot_tsne(X_res, y_res, X_blind, y_blind, mean_x, std_x)